Installing postgres using Homebrew.  
https://www.dataquest.io/blog/install-postgresql-14-7-for-macos/?_gl=1*1vrbtn6*_gcl_au*NjQ1OTI0MzQxLjE3MTYyNzUyMDY.

In [2]:
%load_ext sql

In [3]:
%sql postgresql://postgres@localhost:5432/northwind

In [4]:
%%sql --only use this line if running in Jupyter Notebook
SELECT table_name AS name,
       table_type AS type
  FROM information_schema.tables
 WHERE table_schema = 'public' AND table_type IN ('BASE TABLE', 'VIEW');

 * postgresql://postgres@localhost:5432/northwind
14 rows affected.


name,type
territories,BASE TABLE
order_details,BASE TABLE
employee_territories,BASE TABLE
us_states,BASE TABLE
customers,BASE TABLE
orders,BASE TABLE
employees,BASE TABLE
shippers,BASE TABLE
products,BASE TABLE
categories,BASE TABLE


In [8]:
%%sql -- orders and customers
SELECT 
    O.order_id, O.order_date, O.ship_country, O.customer_id
FROM
    orders O
INNER JOIN customers C ON O.customer_id = C.customer_id
LIMIT 5 

 * postgresql://postgres@localhost:5432/northwind
5 rows affected.


order_id,order_date,ship_country,customer_id
10248,1996-07-04,France,VINET
10249,1996-07-05,Germany,TOMSP
10250,1996-07-08,Brazil,HANAR
10251,1996-07-08,France,VICTE
10252,1996-07-09,Belgium,SUPRD
